# Section 1: Building a Graph-Based RAG Agent with Neo4j and LLM-generated Cyphers

In this section, we will build a **Retrieval-Augmented Generation (RAG)** pipeline that uses a **knowledge graph** (Neo4j) instead of a traditional vector database for retrieval. We'll ingest a corpus of BBC news articles into Neo4j as a graph of connected entities, then use a local large language model (LLM) to translate natural language questions into Cypher queries with `LangChain`'s `GraphCypherQAChain`. The LLM will execute those queries on the graph and generate answers based on the results.

By the end of this lab, you will have a working environment with Neo4j and a local LLM, a Neo4j graph populated with documents, categories, and key entities (with relationships like `BELONGS_TO` and `MENTIONS`), and examples of querying the graph using natural language questions.

## Step 1: Environment Setup

To get started, we need to set up two main components of our environment: a Neo4j graph database and a local LLM for question-answering. We'll use **Docker** to run Neo4j, you will need to download an LLM (we'll provide some recommendations) and set up a Python environment for our code.

### 1.1 Launch Neo4j with Docker

First, spin up a Neo4j instance using Docker. We can use the official Neo4j image and expose the default ports (7474 for HTTP interface, 7687 for Bolt protocol). Below is a **Docker command** that starts a Neo4j instance:

In [ ]:
!pip install udocker
!udocker --allow-root install

!mkdir -p /root/neo4j-inmem
!mount -t tmpfs -o size=1g tmpfs /root/neo4j-inmem
!mkdir -p /root/neo4j-inmem/data
!mkdir -p /root/neo4j-inmem/logs
!mkdir -p /root/neo4j-inmem/import
!mkdir -p /root/neo4j-inmem/plugins

# Run neo4j in docker container
!nohup udocker --allow-root run \
  --publish=7474:7474 --publish=7687:7687 \
  --env NEO4J_AUTH=neo4j/neo4jneo4j \
  -v /root/neo4j-inmem/data:/data \
  -v /root/neo4j-inmem/logs:/logs \
  -v /root/neo4j-inmem/import:/var/lib/neo4j/import \
  -v /root/neo4j-inmem/plugins:/plugins \
  -e NEO4JLABS_PLUGINS='["apoc"]' \
  -e NEO4J_apoc_export_file_enabled=true \
  -e NEO4J_apoc_import_file_enabled=true \
  -e NEO4J_dbms_directories_data=/data \
  neo4j:5.26 &

print("\n\n")
print("Setup Docker Complete!")

### 1.2 Python Environment and Dependencies

With Neo4j running and the model file ready, set up a Python environment for running the ingestion and querying code. You should have Python 3.10+ available. It's recommended to use a virtual environment or Conda environment for the lab.

> **IMPORTANT** This will take ~5-6 minutes to complete on Google Colab.

In [ ]:
!pip install llama-cpp-python neo4j==5.28.1 requests==2.32.3 sentence-transformers==4.1.0 ctransformers==0.2.27 spacy==3.8.5 langchain==0.3.25 langchain-neo4j==0.4.0 langchain-community==0.3.23

!pip install Flask==3.1.0
!pip install gdown==5.2.0

# download small English model for NER
# python -m spacy download en_core_web_sm
import spacy

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

# Download files from my Public Google Drive
print("\n\n")
print("Download Dataset from Google Drive")

import os
from pathlib import Path

import gdown
import zipfile
import shutil

# backup id: 13GRUxdsUUlUK9uC832Su9Qy2mst9Jznq
url = 'https://drive.google.com/uc?id=1f3dqqf9VSnGoVFCP4IozY2AWI39C1UMe'
output = 'bbc-mini.zip'

# Check if the file already exists
if not os.path.exists(output):
    print("Downloading the zip file...")
    gdown.download(url, output, quiet=False)
else:
    print("Zip file already exists, skipping download.")

with zipfile.ZipFile("bbc-mini.zip","r") as zip_ref:
    zip_ref.extractall("./")

print("\n\n")
print("Setup Complete!")

> **IMPORTANT:** Don't move onto the next section until you see a "Complete!" in the output for this section.

### 1.3 Set Up the Local LLM

In this lab, we'll use a 7B parameter model called [neural-chat-7B-v3-3-GGUF](https://huggingface.co/TheBloke/neural-chat-7B-v3-3-GGUF) (a quantized GGUF file). This is the model that will be used in the lab, so for maximum "it just works", stick with this model.

In [ ]:
!wget https://huggingface.co/TheBloke/neural-chat-7B-v3-3-GGUF/resolve/main/neural-chat-7b-v3-3.Q4_K_M.gguf

print("\n\n")
print("Download Complete!")

> **IMPORTANT:** Don't move onto the next section until you see a "Complete!" in the output for this section.

## Step 2: Data Ingestion: From Raw Text to a Queryable Graph

With the environment ready, we'll proceed to prepare our data (BBC articles) and build the knowledge graph in Neo4j as graph nodes and relationships.

Our knowledge source is a collection of BBC news articles in text format which can be found in the zip file [bbc-lite.zip](./workshop/1_llm_cypher/bbc-lite.zip). This zip file ontains a subset of 300 BBC news articles from the 2225 articles in the [BBC Full Text Document Classification](https://bit.ly/4hBKNjp) dataset. After unzipping the archive, the directory structure will look like:

```
bbc/
├── tech/
    ├── 001.txt
    ├── 002.txt
    ├── 003.txt
    ├── 004.txt
    ├── 005.txt
    └── ...
```

Each file is a news article relating to technology in the world today.

### 2.1 What We’re Really Building

Forget vector stores for a moment. We're creating **two node labels** and **one relationship type**-all you need for entity-centric retrieval:

| Node Label       | Key Properties           | Purpose                                    |
| ---------------- | ------------------------ | ------------------------------------------ |
| `:Document`      | `id`, `title`, `content` | Holds the full article text                |
| `:Entity`       | `name`                   | Unique named entities (people, orgs, etc.) |
| **Relationship** | **Direction**            | **Meaning**                                |
| `[:MENTIONS]`    | `(Document) → (Entity)` | "This article talks about that entity."    |

No vectors... just raw NER-driven connections that keep the graph clean and demo-ready.

### 2.2 Ingestion Script

Now we will construct the knowledge graph in Neo4j by creating nodes for **documents** and **entities**, and defining relationships among them. Our graph schema will be:

* **Document** nodes: each article is a document node with properties like `title` (we'll use filename as title) and `content` (the full text).
* **Entity** nodes: significant entities mentioned in the articles (we'll extract these via NER).

Relationships:

* `(:Document)-[:MENTIONS]->(:Entity)` - links a document to an entity it mentions.

We'll use **spaCy** to identify named entities in each article as our "areas of interest." SpaCy's small English model can recognize entities like PERSON, ORG (organization), GPE (location), etc. We'll treat each unique entity text as a Entity node (with an optional property for its type/label).

In [ ]:
#!/usr/bin/env python3

import os
import uuid
import spacy

from neo4j import GraphDatabase

# Neo4j connection settings
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "neo4jneo4j"

# Path to the unzipped BBC dataset folder (with subfolders like 'tech')
DATASET_PATH = "./bbc"

def ingest_bbc_documents_with_ner():
    """
    Ingest BBC documents from the 'technology' subset (or other categories if desired)
    and store them in Neo4j with Document and Entity nodes. The code uses spaCy for NER
    and links documents to extracted entities using MENTIONS relationships.
    """
    # Load spaCy's small English model for Named Entity Recognition
    nlp = spacy.load("en_core_web_sm")

    # Connect to Neo4j
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

    # Perform ingestion in a session
    with driver.session() as session:
        # Optional: clear old data
        print("Clearing old data from Neo4j...")
        session.run("MATCH (n) DETACH DELETE n")
        print("Old data removed.\n")

        # Walk through each category folder
        for category in os.listdir(DATASET_PATH):
            category_path = os.path.join(DATASET_PATH, category)
            if not os.path.isdir(category_path):
                continue  # Skip non-directories

            print(f"Ingesting documents in category '{category}'...")
            for filename in os.listdir(category_path):
                if filename.endswith(".txt"):
                    filepath = os.path.join(category_path, filename)

                    with open(filepath, "r", encoding="utf-8", errors="replace") as f:
                        text_content = f.read()

                    # Generate a UUID for each document
                    doc_uuid = str(uuid.uuid4())

                    # Create or MERGE the Document node
                    create_doc_query = """
                    MERGE (d:Document {doc_uuid: $doc_uuid})
                    ON CREATE SET
                        d.title = $title,
                        d.content = $content,
                        d.category = $category
                    RETURN d
                    """
                    session.run(
                        create_doc_query,
                        doc_uuid=doc_uuid,
                        title=filename,
                        content=text_content,
                        category=category
                    )

                    # Named Entity Recognition
                    doc_spacy = nlp(text_content)

                    # For each recognized entity, MERGE on (name + label)
                    # Then create a relationship from the Document to the Entity.
                    for ent in doc_spacy.ents:
                        # Skip very short or numeric-only entities
                        if len(ent.text.strip()) < 3:
                            continue

                        # Generate a unique ID for new entities
                        entity_uuid = str(uuid.uuid4())

                        merge_entity_query = """
                        MERGE (e:Entity {name: $name, label: $label})
                        ON CREATE SET e.ent_uuid = $ent_uuid
                        RETURN e.ent_uuid as eUUID
                        """
                        record = session.run(
                            merge_entity_query,
                            name=ent.text.strip(),
                            label=ent.label_,
                            ent_uuid=entity_uuid
                        ).single()

                        ent_id = record["eUUID"]

                        # Now create relationship by matching on doc_uuid & ent_uuid
                        rel_query = """
                        MATCH (d:Document { doc_uuid: $docId })
                        MATCH (e:Entity { ent_uuid: $entId })
                        MERGE (d)-[:MENTIONS]->(e)
                        """
                        session.run(
                            rel_query,
                            docId=doc_uuid,
                            entId=ent_id
                        )

            print(f"Finished ingesting category '{category}'.\n")

    driver.close()

# Ingest the data into our RAG pipeline/neo4j
ingest_bbc_documents_with_ner()

print("\n\n")
print("Ingest Complete!")

At this point, we have a rich knowledge graph: documents categorized, and connected to the key entities they mention. This graph can answer more complex questions than a pure vector search - for example, we can traverse from categories to entities to documents, etc., to find multi-hop relationships. We'll leverage this graph for querying in the next step.

> **IMPORTANT:** Don't move onto the next section until you see a "Complete!" in the output for this section.

## Step 3: Querying with LangChain’s GraphCypherQAChain

Now comes the exciting part: using an LLM (the one we set up in Step 1) to query the Neo4j graph with natural language. LangChain provides a chain specifically for this purpose called **GraphCypherQAChain**. This chain integrates an LLM with a graph by having the **LLM generate Cypher queries** in response to user questions, retrieving data from the graph, and then formulating a final answer. In other words, the LLM acts as a translator from English to Cypher and then uses the query results to compose an answer.

We will configure LangChain's GraphCypherQAChain to use:

* Our **local LLM** as the language model that will do the reasoning and query generation.
* A **Neo4j graph** connection (pointing to our populated database) for executing the Cypher queries.

### 3.1 Hook Up LangChain to Neo4j and the Local LLM

This code will perform the RAG Query using our Graph-based implementation on Neo4j. Based on the data ingested, we will ask the RAG Agent the following questions:

- What are the top 5 most mentioned entities in these articles?

> **IMPORTANT:** Running the script below will take approximately 4-5 minutes.

In [ ]:
#!/usr/bin/env python3

import os
import sys
import time

from langchain_community.graphs import Neo4jGraph
from langchain_community.llms import LlamaCpp
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

# ────────────────────────────── Neo4j Settings ───────────────────────────────
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASS = os.getenv("NEO4J_PASS", "neo4jneo4j")

# ───────────────────────────── llama-cpp Settings ────────────────────────────
MODEL_PATH = "./neural-chat-7b-v3-3.Q4_K_M.gguf"  # Adjust to your local model

# ─────────────────────────────────────────────────────────────────────────────
# Prompt template for generating Cypher queries only. We are instructing the LLM
# to return a single valid Cypher query.
# ─────────────────────────────────────────────────────────────────────────────
CYTHER_ONLY_PROMPT = PromptTemplate(
    input_variables=["schema", "query"],
    template=(
        "You are an expert in Neo4j Cypher.\n"
        "Graph schema:\n{schema}\n\n"
        "Given a natural-language question, return ONE valid Cypher query "
        "that answers it.\n"
        "Output **only** the Cypher query—no explanation, no labels, no "
        "markdown fences.\n\n"
        "{query}"
    ),
)

def wait_for_neo4j(uri: str, user: str, pwd: str, tries: int = 10, delay: int = 3):
    """
    Ping the DB until it responds to 'RETURN 1'. This ensures the DB is up
    before the script attempts to run queries.
    """
    for i in range(1, tries + 1):
        try:
            graph = Neo4jGraph(url=uri, username=user, password=pwd)
            graph.query("RETURN 1")
            print(f"✓ Neo4j is ready on {uri}")
            return
        except Exception as e:
            print(f"[{i}/{tries}] Neo4j not reachable ({e}); retrying in {delay}s")
            time.sleep(delay)

    sys.exit("Neo4j never came online. Exiting.")


def main():
    # 1) Ensure the Neo4j database is reachable
    wait_for_neo4j(NEO4J_URI, NEO4J_USER, NEO4J_PASS)

    # 2) Load the graph (this uses the Neo4jGraph class from langchain_community)
    graph = Neo4jGraph(
        url=NEO4J_URI,
        username=NEO4J_USER,
        password=NEO4J_PASS,
        # enhanced_schema=True tries to infer node labels/relationships automatically
        enhanced_schema=True,
    )
    print("Detected schema:\n", graph.schema, "\n")

    # 3) Load local llama-cpp model
    llm = LlamaCpp(
        model_path=MODEL_PATH,
        n_ctx=32768,
        n_threads=8,
        temperature=0.2,
        top_p=0.95,
        repeat_penalty=1.2,
        verbose=False,
    )

    # 4) Build the Cypher-aware QA chain
    chain = GraphCypherQAChain.from_llm(
        llm,
        graph=graph,
        cypher_prompt=CYTHER_ONLY_PROMPT,    # Our specialized prompt
        validate_cypher=True,                # Validate the query
        verbose=False,
        allow_dangerous_requests=True,       # Allows MERGE if needed
    )

    # 5) Example questions for demonstration
    questions = [
        "What are the top 5 most mentioned entities in these articles?",
    ]

    # 6) Perform queries and display results
    for q in questions:
        print(f"\nQ: {q}")
        result = chain.invoke({"query": q})["result"]  # key must be "query"
        print("A:", result)


# run the RAG query
main()
print("\n\n")
print("RAG Complete!")

A couple of notes:

* `Neo4jGraph` is a LangChain wrapper that uses the Neo4j Python driver under the hood. It can optionally introspect the database to understand the schema. If `enhanced_schema=True`, it will sample some data to list example property values which can help the LLM understand the domain (this can be useful, though not strictly required).
* We printed `graph.schema` to see what it detected. It should list labels like **Document**, **Entity** and their properties (e.g., `Document` has properties `id`, `title`, `content`, etc.) and relationships like **MENTIONS**. This schema info will be provided to the LLM in its prompt context so that it knows how to form the Cypher queries.
* We then load the LLM. We use `Llama` from LangChain, giving the path to our .gguf model file. We also specify `n_ctx=32768` to set a context window (adjust based on model capability). If you have GPU acceleration, you could pass parameters like `n_gpu_layers` or use a different BLAS, but CPU should work for a 7B model (albeit slowly). The `verbose=False` just suppresses internal logging from the LLM.

#### Understanding the LLM-Generated Cypher Queries

It's worth noting that **we did not manually program any Cypher queries** for our Q&A - the LLM generated them on the fly based on the question and the graph schema. This demonstrates a powerful pattern:

* The **knowledge graph** stores facts and relationships explicitly (documents, their topics, categories, etc.).
* The **LLM** acts as a reasoner and translator, mapping a natural question to the right graph query, and then interpreting the results.

LangChain's GraphCypherQAChain provided the scaffolding to make this happen easily. If you check the `verbose` output, you might see something like:

```
> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Document)-[:MENTIONS]->(p:Entity)
RETURN p.name, count(*) ORDER BY count(*) DESC LIMIT 5
Full Context:
[{'p.name': 'government', 'count(*)': 12}, {'p.name': 'Prime Minister', 'count(*)': 8}, ...]
> Finished chain.
```

> **IMPORTANT:** It is also worth noting that this method is **EXTREMELY** brittle. Try changing the questions by rephrasing. How did it fair? Not good I am guessing.

# Section 2: Building a Graph-Based RAG Agent with Neo4j and Fixed Path Cyphers

Now, let's dive into a powerful and efficient alternative approach: using predefined, or **fixed Cypher paths**, to interact with your Neo4j graph database within a Retrieval-Augmented Generation (RAG) agent. Rather than dynamically generating queries on-the-fly with an LLM, this method involves setting up specific, fixed Cypher queries optimized for common retrieval tasks. These carefully crafted queries ensure consistent performance, predictable results, and streamlined interactions with the database.

In this section, you'll learn how to:

* Define and utilize **fixed Cypher queries** designed for frequent and high-impact retrieval scenarios.
* Connect these fixed paths directly into your RAG Agent workflow, enabling rapid responses to user questions without the overhead of dynamic query generation.

By leveraging fixed Cypher paths, your RAG agent maintains efficiency, reduces complexity, and delivers reliable, lightning-fast answers tailored precisely to your application's needs.

### Step 1: Using Precise/Fixed Cypher Paths Has MANY Benefits

This code will perform the RAG Query using our Graph-based implementation on Neo4j. Based on the data ingested, we will ask the RAG Agent the following questions:

- What do these articles say about Ernie Wise?

In [ ]:
#!/usr/bin/env python3
"""
RAG pipeline (Neo4j + spaCy + Llama-cpp)
Model tested with TheBloke/neural-chat-7B-v3-3.Q4_K_M.gguf
"""

import os
from functools import lru_cache

import spacy
from neo4j import GraphDatabase
from llama_cpp import Llama

##############################################################################
# Neo4j connection details
##############################################################################

NEO4J_URI      = os.getenv("NEO4J_URI",      "bolt://localhost:7687")
NEO4J_USER     = os.getenv("NEO4J_USER",     "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

##############################################################################
# Llama-cpp configuration
##############################################################################

MODEL_PATH      = os.getenv("MODEL_PATH", "./neural-chat-7b-v3-3.Q4_K_M.gguf")

@lru_cache(maxsize=1)
def load_llm() -> Llama:
    """
    Load the GGUF model once, cache, and reuse.
    """
    print(f"Loading model from {MODEL_PATH} …")
    return Llama(
        model_path=MODEL_PATH,
        n_ctx=32768,
        temperature=0.2,
        top_p=0.95,
        repeat_penalty=1.2,
        verbose=False,
        chat_format="chatml",  # Neural-Chat uses the ChatML template
    )

    # use_gpu=True,
    # n_gpu_layers=-1,      # offload *all* transformer layers to the GPU
    # n_threads=2,          # spawn enough CPU threads to feed the GPU
    # n_batch=256,          # process 256 tokens at once for throughput
    # f16_kv=True,          # store KV cache in half-precision on GPU


##############################################################################
# Neo4j helpers
##############################################################################

def connect_neo4j():
    return GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

##############################################################################
# spaCy Named-Entity Recognition
##############################################################################

def extract_entities_spacy(text, nlp):
    doc = nlp(text)
    return [(ent.text.strip(), ent.label_) for ent in doc.ents if len(ent.text.strip()) >= 3]

##############################################################################
# Graph query - fetch docs mentioning entities
##############################################################################

def fetch_documents_by_entities(session, entity_texts, top_k=5):
    if not entity_texts:
        return []

    query = """
    MATCH (d:Document)-[:MENTIONS]->(e:Entity)
    WHERE toLower(e.name) IN $entity_list
    WITH d, count(e) as matchingEntities
    ORDER BY matchingEntities DESC
    LIMIT $topK
    RETURN d.title AS title, d.content AS content,
           d.category AS category, matchingEntities
    """
    entity_list_lower = [txt.lower() for txt in entity_texts]

    results = session.run(query,
                          entity_list=entity_list_lower,
                          topK=top_k)

    docs = []
    for r in results:
        docs.append({
            "title":  r["title"],
            "content": r["content"],
            "category": r["category"],
            "match_count": r["matchingEntities"]
        })
    return docs

##############################################################################
# LLM-based answer generation
##############################################################################

def generate_answer(question: str, context: str) -> str:
    llm = load_llm()

    system_msg = "You are an expert assistant answering questions using the given context."
    user_prompt = (
        f"You are given the following context from multiple documents:\n"
        f"{context}\n\nQuestion: {question}\n\nProvide a concise answer."
    )

    response = llm.create_chat_completion(
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_prompt},
        ],
        temperature=0.2,
        top_p=0.95,
        max_tokens=32768,
    )
    return response["choices"][0]["message"]["content"].strip()

##############################################################################
# Main
##############################################################################

user_query = "What do these articles say about Ernie Wise?"
print(f"User Query: {user_query}")

# Load spaCy model once
nlp = spacy.load("en_core_web_sm")

# NER over user query
recognized_entities = extract_entities_spacy(user_query, nlp)
entity_texts = [ent[0] for ent in recognized_entities]
print("Recognized entities:", recognized_entities)

# Neo4j — fetch docs
driver = connect_neo4j()
with driver.session() as session:
    docs = fetch_documents_by_entities(session, entity_texts, top_k=5)

# Build context
combined_context = ""
for doc in docs:
    snippet = doc["content"][:300].replace("\n", " ")
    combined_context += (
        f"\n---\nTitle: {doc['title']} | Category: {doc['category']}\n"
        f"Snippet: {snippet}...\n"
    )

# Ask the model
final_answer = generate_answer(user_query, combined_context)
print("\nRAG-based Answer:\n", final_answer)

print("\n\n")
print("RAG Complete!")

Here are four key considerations when using and extending this RAG pipeline:

1. **Context Window & Token Limits**

   * You must stay within the your LLMs context length (`n_ctx`) and the `max_tokens` you request. If your combined document snippets exceed this window, the model may truncate earlier context, reducing answer quality. The larger the context window, the better!
   * Consider summarizing or chunking very long documents before passing them in.

2. **Secure, Efficient Cypher Queries**

   * Always use parameterized Cypher (as shown) to prevent injection attacks and leverage Neo4j's query planning.
   * If you add an `expiration` filter on `[:MENTIONS]`, embed timestamps as parameters so Neo4j can cache and reuse the query plan.

3. **NER Accuracy & Coverage**

   * The small spaCy model (`en_core_web_sm`) is fast but may miss or mislabel domain-specific entities. For technical content, consider a larger or custom-trained NER model.
   * Always normalize (e.g. `toLower()`) both your stored entity names and extracted entities to improve matching.

#### 1.1 Understanding the Fixed Cypher Path Queries

In contrast to LLM-generated queries, **we explicitly define every Cypher statement** up front, mapping each question pattern to a predetermined graph traversal. This "fixed path" approach highlights a different but equally powerful pattern:

* The **knowledge graph** still holds all facts and relationships explicitly (documents, entities, categories, etc.).
* The **developer** now acts as the translator, hand-crafting precise Cypher templates that retrieve exactly the intended nodes and relationships.

For example, consider a fixed query to fetch all documents mentioning a given entity:

```cypher
MATCH (d:Document)-[:MENTIONS]->(e:Entity {name: $entityName})
RETURN
  d.title      AS title,
  d.category   AS category,
  substring(d.content, 0, 200) AS snippet
LIMIT $limit
```

When executed in Python:

```python
fixed_query = """
MATCH (d:Document)-[:MENTIONS]->(e:Entity {name: $entityName})
RETURN d.title AS title, d.category AS category,
       substring(d.content,0,200) AS snippet
LIMIT $limit
"""
params = {"entityName": "quantum computing", "limit": 5}
results = session.run(fixed_query, **params)
for r in results:
    print(f'Title: "{r["title"]}" | Category: "{r["category"]}"\nSnippet: {r["snippet"]}…')
```

You might see output such as:

```
Title: "Quantum Leap in Computing"   | Category: "tech"
Snippet: "Researchers at Google have unveiled a new 72-qubit quantum processor…"…

Title: "Advances in Quantum Hardware" | Category: "science"
Snippet: "A team at IBM demonstrated error correction on a superconducting quantum…"…
```

This fixed-path method offers clear benefits:

* **Predictable Performance**: Neo4j can cache and optimize these well-known queries.
* **Deterministic Results**: Each question corresponds to a single, auditable Cypher template.
* **Easier Maintenance & Security**: Queries are defined in code, making it simpler to review, test, and secure against injection.

By combining fixed Cypher paths with RAG-style context assembly (or even alongside LLM-generated queries), you gain full control over both performance and flexibility in your graph-powered Q&A system.

# Workshop End!

This covers using both LLM-generated Cypher Paths using GraphCypherQAChain and Fixed Cypher Paths. Let's move onto the next section to understand how Reinforcement Learning works in a Graph-based RAG implementations!